<div class="alert alert-block alert-info">
<span style="color: rgb(0,53,91);">
<center><img src="https://www.redage.org/sites/default/files/styles/large/public/img-logo-institucion/logo_iteso.jpg?itok=IzloBJS1" style="width:500px;height:142px;" title="Logo ITESO"></center>

<font face = "Times New Roman" size = "6"><b><center>Programacion para Mineria de Datos</center></b></font>
<font face = "Times New Roman" size = "5"><b><center>Preprocesamiento PCA</center></b></font>

<b><font back = "Times New Roman" size = "4"><center>Examen Parcial 3</center></font>
<div align="right"><font face = "Times New Roman" size = "2">nombres: Angel Aceves Manzo</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jorge Arturo Ponce Martin</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jesus Vargas Pacheco</font></div>
</span></div>

In [1]:
import pandas as pd
import joblib, json
import numpy as np

In [2]:
##################################################################################################
Train = pd.read_csv("../01_preprocessing_results/preprocessing/T_train_final_objetivo.csv")
Test = pd.read_csv("../01_preprocessing_results/preprocessing/T_test_final_objetivo.csv")
##################################################################################################

In [3]:
X_train = Train.iloc[:, :-1]
y_train = Train.iloc[:, -1].to_numpy(dtype=float)

X_test = Test.iloc[:, :-1]
y_test = Test.iloc[:, -1].to_numpy(dtype=float)

In [4]:
modelo = joblib.load("../02_lineal_regression_results/regression_lineal/modelo_reg_lineal.pkl")

In [5]:
intercepto = modelo.named_steps["linreg"].intercept_
feature_names = modelo.named_steps["dropper"].get_feature_names_out(X_train.columns)
coefs = modelo.named_steps["linreg"].coef_

In [6]:
print(f"intercepto:{intercepto}")
print(f"feature_names:{feature_names}")
print(f"coefs:{coefs}")

intercepto:236960.31209482663
feature_names:['PC1' 'PC2' 'PC3' 'PC4' 'PC5' 'PC6' 'PC7' 'PC8' 'heating___hot air'
 'heating___hot water/steam' 'fuel___gas' 'fuel___oil'
 'sewer___public/commercial' 'sewer___septic' 'waterfront___Yes'
 'newConstruction___Yes' 'centralAir___Yes']
coefs:[ 39652.62218781  -6248.34152754   3416.68103819   4415.11462984
  31651.76429274  -1343.95706409   9256.9429326   -9242.38590697
   4309.0533833  -11432.48364879   4081.70488621    647.06600336
 -30736.54203619 -25361.29307722 116692.46163681 -43289.65382166
   6810.96276312]


In [7]:
# === Betas en espacio post-`pre` (antes del StandardScaler de la PCA) ===

# 1) Cargar el pipeline numérico usado en PCA_full
########################################################################################
pca_pipe = joblib.load("../01_preprocessing_results/preprocessing/pca_pipe_num.joblib")
########################################################################################
pre = pca_pipe.named_steps["pre"]  # ColumnTransformer (pre)
pre

,transformers,"[('num_med_rob', ...), ('num_mean_min', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [8]:
scaler = pca_pipe.named_steps["std_for_pca"]  # StandardScaler
scaler

,copy,True
,with_mean,True
,with_std,True


In [9]:
pca = pca_pipe.named_steps["pca"]  # PCA(n_components=k)
pca

,n_components,8
,copy,True
,whiten,False
,svd_solver,'full'
,tol,0.0
,iterated_power,'auto'
,n_oversamples,10
,power_iteration_normalizer,'auto'
,random_state,0


In [10]:
# 2) Extraer objetos necesarios
pre_feature_names = pre.get_feature_names_out()  # nombres de las p numéricas post-`pre`
pre_feature_names

array(['landValue', 'age', 'lotSize', 'livingArea', 'pctCollege',
       'bedrooms', 'fireplaces', 'bathrooms', 'rooms'], dtype=object)

In [11]:
mu = scaler.mean_  # (p,) Medias del EstandardScaler
mu

array([3.92993051e-01, 4.30752736e-01, 5.58616714e-02, 2.49589243e-01,
       5.72818573e-01, 8.82661784e-17, 4.96497253e-17, 1.35157586e-16,
       1.46190858e-16])

In [12]:
print(sorted(mu,reverse=True))

[0.5728185734321779, 0.4307527358769595, 0.39299305085788083, 0.2495892432918716, 0.055861671409875566, 1.461908579009523e-16, 1.351575856065408e-16, 8.826617835529194e-17, 4.964972532485172e-17]


In [13]:
sig = scaler.scale_  # (p,) Desviaciones Estandar del StandardScaler
sig

array([1.43130047, 1.40916918, 0.0746441 , 0.13727138, 0.1661593 ,
       1.        , 1.        , 1.        , 1.        ])

In [14]:
print(sorted(sig))

[0.07464409576737213, 0.13727137552040944, 0.16615930226833103, 0.9999999999999999, 0.9999999999999999, 0.9999999999999999, 1.0, 1.4091691759224387, 1.4313004689136515]


In [15]:
C = pca.components_  # (k, p) Matriz de Componentes Principales (EigenVectores)
C

array([[ 0.27364411, -0.12714938,  0.09459072,  0.49063879,  0.17217498,
         0.39327072,  0.32643765,  0.42703892,  0.4313331 ],
       [-0.11074924,  0.65935977,  0.39973242,  0.07736183, -0.31889114,
         0.36403112, -0.21920985, -0.20368353,  0.25190927],
       [ 0.36610116,  0.53839212, -0.44943289, -0.0314508 ,  0.57802214,
         0.04002359, -0.00475139, -0.19300932, -0.01175327],
       [ 0.17140792,  0.03112635,  0.77663099, -0.08521695,  0.4371675 ,
        -0.22055331,  0.24262336, -0.11171356, -0.21938015],
       [ 0.8211763 , -0.00924434,  0.05534396,  0.06672905, -0.43798572,
        -0.26852312, -0.22317672,  0.03365703, -0.05649339],
       [-0.0263388 ,  0.31766337, -0.14357577, -0.00446684, -0.38187482,
        -0.14862358,  0.82384684, -0.01281844, -0.17594607],
       [-0.18336159,  0.38371251,  0.02868476,  0.1550348 ,  0.05272856,
        -0.267823  , -0.22599989,  0.75722915, -0.30871223],
       [ 0.15467096, -0.09015945,  0.00407696, -0.11013904, -0

In [16]:
B = pca.explained_variance_  #Varianza Explicada por Valores Propios
B

array([3.51394439, 1.16734391, 1.05295185, 0.91729866, 0.78685646,
       0.67051925, 0.40555996, 0.31780453])

In [17]:
# 3) Aislar betas de Componentes Principales del modelo (según prefijo)
pc_prefix = "PC"
# Creación de la máscara para componentes principales
pc_mask = np.array([c.startswith(pc_prefix) for c in feature_names], dtype=bool)

# Extraccion de coeficientes de componentes principales
beta_PC = coefs[pc_mask]  # (k,)
 
# 4) Transformacion inversa: PCs -> post-`pre` (Variables Estandarizadas
gamma = C.T @ beta_PC
 
# Transformación: Variables estandarizadas → Variables originales
alpha_pre = gamma / sig  # betas por cada variable numérica post-`pre` (p,)

#Ajuste del Intercepto
beta0_pre = intercepto - np.sum(gamma * (mu / sig))  # intercepto en el mismo espacio
 
# 5) Empaquetar resultados
# Betas despues del Post Procesamiento
betas_post_pre_df = pd.DataFrame({
    "feature_post_pre": pre_feature_names,
    "beta_post_pre": alpha_pre
})
########################################################################################################
with open("../01_preprocessing_results/preprocessing/pca_metadata.json", "r", encoding="utf-8") as f:
#######################################################################################################
    meta = json.load(f)
 
orden_original_num = meta["cols_num"]  # orden original de columnas numéricas
presentes = set(betas_post_pre_df["feature_post_pre"].tolist())
orden_final = [c for c in orden_original_num if c in presentes]
 
betas_post_pre_df = (
    betas_post_pre_df
    .set_index("feature_post_pre")
    .loc[orden_final]
    .reset_index()
)
 
print("\nIntercepto (post `pre`):", beta0_pre)
print("\nBetas numéricas (post `pre`, en orden original):")
print(betas_post_pre_df)


Intercepto (post `pre`): 180881.77024539228

Betas numéricas (post `pre`, en orden original):
  feature_post_pre  beta_post_pre
0          lotSize   71258.111382
1              age   -2497.218569
2        landValue   25466.716038
3       livingArea  167982.746134
4       pctCollege    2162.463840
5         bedrooms   -4709.128682
6       fireplaces    5146.947453
7        bathrooms   24451.207090
8            rooms   16419.232586


In [18]:
# === Predicción en TEST ===
yhat_test = modelo.predict(X_test)
 
# Métricas básicas en test
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
 
r2_test   = r2_score(y_test, yhat_test)
rmse_test = math.sqrt(mean_squared_error(y_test, yhat_test))
mae_test  = mean_absolute_error(y_test, yhat_test)
 
print({"R2_test": r2_test, "RMSE_test": rmse_test, "MAE_test": mae_test})
 
# También en train, para comparar generalización
yhat_train = modelo.predict(X_train)
r2_train   = r2_score(y_train, yhat_train)
rmse_train = math.sqrt(mean_squared_error(y_train, yhat_train))
mae_train  = mean_absolute_error(y_train, yhat_train)
print({"R2_train": r2_train, "RMSE_train": rmse_train, "MAE_train": mae_train})

{'R2_test': 0.6366758627712676, 'RMSE_test': 53869.7671822696, 'MAE_test': 40406.711645091185}
{'R2_train': 0.6405260393241826, 'RMSE_train': 60675.91817438013, 'MAE_train': 42847.82894146506}


In [19]:
# Veredicto

# NRMSE y mejora vs media en TEST
std_y_test = float(np.std(y_test, ddof=0))
nrmse_test = rmse_test / (std_y_test + 1e-12)
mejora_pct = 100.0 * (1.0 - nrmse_test)  # % mejor que predecir la media

# Semáforo (sin CV), "aquí dentro"
verde   = (r2_test >= 0.70) and (nrmse_test <= 0.50)
amarilo = (0.40 <= r2_test < 0.70) or (0.50 < nrmse_test <= 0.80)
if verde:
    veredicto = "VERDE"
    significado = "confiable para predecir aquí dentro."
elif amarilo:
    veredicto = "AMARILLO"
    significado = "usable con cautela (depende del caso de uso)"
else:
    veredicto = "ROJO"
    significado = "no confiable para predicción aquí dentro"

# Generar explicación humanizada automáticamente
def generar_explicacion(r2_test, nrmse_test, mejora_pct, rmse_test, mae_test, veredicto):
    explicacion = []

    # Explicación basada en R²
    if r2_test >= 0.8:
        explicacion.append(f"• El modelo explica un {r2_test:.1%} de la variabilidad en los datos, lo que indica un excelente ajuste.")
        explicacion.append("  🎯 **Analogía**: Como un pronóstico del tiempo que acierta 8 de cada 10 días - muy confiable.")
    elif r2_test >= 0.6:
        explicacion.append(f"• El modelo captura un {r2_test:.1%} de la variabilidad, mostrando una buena capacidad predictiva.")
        explicacion.append("  🎯 **Analogía**: Similar a un forecast económico que identifica correctamente las tendencias principales.")
    elif r2_test >= 0.4:
        explicacion.append(f"• Con un R² del {r2_test:.1%}, el modelo tiene capacidad predictiva moderada.")
        explicacion.append("  🎯 **Analogía**: Como un detector de lluvia que funciona bien para saber si lloverá, pero no cuánto.")
    else:
        explicacion.append(f"• El R² de {r2_test:.1%} sugiere que el modelo tiene capacidad predictiva limitada.")
        explicacion.append("  🎯 **Analogía**: Parecido a adivinar el clima lanzando una moneda - mejor que nada, pero poco confiable.")

    # Explicación basada en NRMSE
    if nrmse_test <= 0.3:
        explicacion.append(f"• Los errores de predicción son muy bajos ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Como medir con una regla milimetrada - alta precisión en las estimaciones.")
    elif nrmse_test <= 0.5:
        explicacion.append(f"• Los errores son moderados ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Similar a usar una cinta métrica - útil para la mayoría de propósitos prácticos.")
    elif nrmse_test <= 0.7:
        explicacion.append(f"• Los errores son considerables ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Como estimar distancias a ojo - sirve para aproximaciones gruesas.")
    else:
        explicacion.append(f"• Los errores son muy altos ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Parecido a adivinar el tamaño de algo desde lejos - muy impreciso.")

    # Explicación basada en mejora vs media
    if mejora_pct > 50:
        explicacion.append(f"• Es un {mejora_pct:.0f}% mejor que simplemente predecir el promedio, una mejora sustancial.")
        explicacion.append("  🚀 **Analogía**: Como usar GPS vs. solo un mapa de carreteras - mucho más eficiente.")
    elif mejora_pct > 20:
        explicacion.append(f"• Mejora en un {mejora_pct:.0f}% respecto a predecir la media.")
        explicacion.append("  🚀 **Analogía**: Similar a tener indicaciones de tráfico en tiempo real - claramente mejor que sin ellas.")
    elif mejora_pct > 0:
        explicacion.append(f"• Solo un {mejora_pct:.0f}% mejor que predecir el promedio, mejora marginal.")
        explicacion.append("  🚀 **Analogía**: Como tener una brújula en lugar de solo el norte - ayuda, pero no demasiado.")
    else:
        explicacion.append("• No mejora respecto a predecir el valor promedio.")
        explicacion.append("  🚀 **Analogía**: Como intentar navegar sin brújula ni mapa - no aporta ventaja.")

    # Comparación entre train y test (si estuvieran disponibles ambos)
    if 'r2_train' in locals():
        sobreajuste = r2_train - r2_test
        if sobreajuste > 0.2:
            explicacion.append(f"• Hay indicios de sobreajuste (R² train: {r2_train:.3f} vs test: {r2_test:.3f}).")
            explicacion.append("  ⚠️ **Analogía**: Como un estudiante que memoriza las respuestas pero no entiende el concepto.")
        elif sobreajuste < 0.05:
            explicacion.append("• El modelo generaliza bien, sin signos evidentes de sobreajuste.")
            explicacion.append("  ✅ **Analogía**: Similar a un atleta que entrena y compite igual de bien - consistente.")

    # Interpretación del veredicto con analogías finales
    if veredicto == "VERDE":
        explicacion.append("\n✅ **Conclusión**: El modelo es confiable para hacer predicciones en contextos similares a los datos de prueba.")
        explicacion.append("🎯 **Analogía final**: Como un piloto automático confiable - puedes usarlo para navegar con seguridad.")
    elif veredicto == "AMARILLO":
        explicacion.append("\n⚠️ **Conclusión**: Úsalo con precaución - puede ser útil para identificar tendencias pero no para predicciones precisas.")
        explicacion.append("🎯 **Analogía final**: Como el forecast de fin de semana - útil para planear, pero lleva paraguas por si acaso.")
    else:
        explicacion.append("\n❌ **Conclusión**: Se recomienda revisar las variables o considerar modelos alternativos.")
        explicacion.append("🎯 **Analogía final**: Como un mapa muy antiguo - mejor conseguir uno actualizado o usar otros métodos.")

    return "\n".join(explicacion)

# Generar la explicación
explicacion_humanizada = generar_explicacion(
    r2_test, nrmse_test, mejora_pct, rmse_test, mae_test, veredicto
)

resumen = {
    "R2_test": r2_test,
    "MAE_test": mae_test,
    "RMSE_test": rmse_test,
    "std(y_test)": std_y_test,
    "NRMSE_test": nrmse_test,
    "Mejora_vs_media_%": mejora_pct,
    "Veredicto": veredicto,
    "significado": significado
}

df_resumen = pd.DataFrame(resumen, index=[0]).T
df_resumen.columns = ["Valor"]
df_resumen.index.name = "Métrica"

print(df_resumen)
print("\n" + "="*60)
print("EXPLICACIÓN DEL VEREDICTO:")
print("="*60)
print(explicacion_humanizada)

def explicacion_breve(r2_test, nrmse_test, mejora_pct, veredicto):
    base = f"Con un R² de {r2_test:.3f} y un error relativo (NRMSE) de {nrmse_test:.3f}, "

    # Añadir analogías a la versión breve también
    if veredicto == "VERDE":
        analogia = "Como un GPS confiable - puedes seguir sus indicaciones con seguridad."
        return base + f"el modelo es robusto y explica bien los patrones en los datos, siendo {mejora_pct:.0f}% mejor que usar promedios simples. {analogia}"
    elif veredicto == "AMARILLO":
        analogia = "Similar a un pronóstico de lluvia - útil para planear, pero lleva sombrilla por si acaso."
        return base + f"el modelo tiene capacidad predictiva limitada ({mejora_pct:.0f}% mejor que promedios), adecuado para análisis exploratorios pero no para decisiones críticas. {analogia}"
    else:
        analogia = "Como un mapa desactualizado - mejor buscar herramientas más precisas."
        return base + f"el modelo no supera significativamente las predicciones básicas ({mejora_pct:.0f}% mejora), recomendando revisar el enfoque. {analogia}"

print("\n" + "🔍 RESUMEN INTERPRETATIVO:")
print("="*40)
print(explicacion_breve(r2_test, nrmse_test, mejora_pct, veredicto))

                                                          Valor
Métrica                                                        
R2_test                                                0.636676
MAE_test                                           40406.711645
RMSE_test                                          53869.767182
std(y_test)                                        89371.278984
NRMSE_test                                             0.602764
Mejora_vs_media_%                                     39.723625
Veredicto                                              AMARILLO
significado        usable con cautela (depende del caso de uso)

EXPLICACIÓN DEL VEREDICTO:
• El modelo captura un 63.7% de la variabilidad, mostrando una buena capacidad predictiva.
  🎯 **Analogía**: Similar a un forecast económico que identifica correctamente las tendencias principales.
• Los errores son considerables (60.3% de la variabilidad total).
  📏 **Analogía**: Como estimar distancias a ojo - sirve para aproxim

In [20]:
###### Guardado del modelo
 
import joblib, json, time
 
# guarda el pipeline completo (dropper + LinearRegression)
joblib.dump(modelo, "modelo_reg_lineal.pkl")
 
# guarda el orden/esperado de columnas de entrenamiento
expected_cols = X_train.columns.tolist()
with open("expected_columns.json", "w", encoding="utf-8") as f:
    json.dump({"columns": expected_cols, "saved_at": time.strftime("%Y-%m-%d %H:%M:%S")}, f)
 
print("Artefactos guardados:", "modelo_reg_lineal.pkl", "expected_columns.json")

Artefactos guardados: modelo_reg_lineal.pkl expected_columns.json


In [21]:
import os, zipfile, glob
 
# Carpeta destino en tu PC
dst_dir = r"mi_regresion_lineal"
os.makedirs(dst_dir, exist_ok=True)
zip_path = os.path.join(dst_dir, "mi_reg_lin_artifacts_bundle.zip")
 
# Archivos que quieres incluir (ajusta si te falta alguno)
candidates = [
    "modelo_reg_lineal.pkl",
    "expected_columns.json",
]
 
present = [f for f in candidates if os.path.exists(f)]
# Si quieres incluir una carpeta (p. ej., 'sample_data'), descomenta:
# for root, _, files in os.walk("sample_data"):
#     for f in files:
#         present.append(os.path.join(root, f))
 
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for f in present:
        zf.write(f, arcname=os.path.basename(f))  # guarda sin subcarpetas
 
print("ZIP creado en:", zip_path)
print("Incluidos:", present)

ZIP creado en: mi_regresion_lineal\mi_reg_lin_artifacts_bundle.zip
Incluidos: ['modelo_reg_lineal.pkl', 'expected_columns.json']
